### Basic LangGraph Chatbot

In [1]:
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from langgraph.graph import START, StateGraph,END
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from config import settings
from langchain.vectorstores import Chroma
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
llm = OllamaLLM(model="mistral")


In [3]:
llm

OllamaLLM(model='mistral')

In [4]:
# def chatbot_node(state: State) -> State:
#     """ The main chatbot node that processes the user input and returns the response """
#     # print(f"Processing {state['messages']} messages")
#     response =  llm.invoke(state['messages'])
#     print(f"Response: {response}")
#     return {"messages": state['messages']}

# print("Chatbot node function created")

In [5]:
# --- Create the retriever BEFORE the graph ---
def get_retriever():
    embeddings = OllamaEmbeddings(model=settings.EMBEDDING_MODEL, base_url=settings.OLLAMA_BASE_URL)
    loader = TextLoader(settings.DATA_FILE)
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)

    persist_directory = os.path.abspath("chroma_db")

    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name= 'rag-chroma',
        persist_directory=persist_directory,
    )

    print("Retriever created")
    return vectorstore.as_retriever()

retriever = get_retriever()  # Create once and reuse


Retriever created


In [6]:
# graph_builder = StateGraph(State)
# graph_builder.add_node("retrieve", split_documents)
# graph_builder.add_node("chatbot", chatbot_node)
# graph_builder.add_edge(START, "retrieve")
# graph_builder.add_edge("retrieve", "chatbot")
# graph_builder.add_edge("chatbot", END)

# graph = graph_builder.compile()

# print("Graph structure created")
# print("Graph compiled successfully")
# graph.invoke({"messages": [HumanMessage(content="How secure is financial data in GL?")]})
# # graph.invoke({"messages": "How secure is financial data in GL?"})


In [14]:
img= graph.get_graph(xray=True).draw_mermaid_png()
with open("graph.png", "wb") as f:
    f.write(img)
from IPython.display import Image
display(Image("graph.png"))

NameError: name 'graph' is not defined

### TESTING THE CHATBOT

In [ ]:
# def test_chatbot(message: str):
#     """Helper function to test the chatbot"""
#     initial_state={"messages": [HumanMessage(content=message)]}
#     print("svsdv",initial_state)
#     response = graph.invoke(initial_state)
#     ai_response = response["messages"][-1].content
#     print("AI:", ai_response)
#     # print("Response:", response)
#     return response
# test_cases = [
#     "Hello my name is Ahmad",
#     "Do you know who I am?",
# ]
# for test_case in test_cases:
#     test_chatbot(test_case)


### ADDING MEMORY TO THE CHATBOT

In [ ]:
# memory = MemorySaver()

# graph_with_memory = graph_builder.compile(checkpointer=memory)

# # print("Memory added to the graph")

# def chat_with_meomry(message: str, thread_id:str):
#     """ Chat function with memory """
#     print("User:", message)

#     config = {
#         "configurable": {
#             "thread_id": thread_id
#         }
#     }
#     initial_state = {"messages": [HumanMessage(content=message)]}
#     result = graph_with_memory.invoke(initial_state, config)
#     ai_response = result["messages"][-1].content
#     # print("AI:", ai_response)
#     # print("Memory:",result)
#     return result

# chat_with_meomry("Hi my name is Ahmad", "thread-1")
# chat_with_meomry("What is my name?", "thread-1")


In [15]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    context: str  # Or list[str] if multiple docs



def retrieve_node(state: State) -> State:
    query = state["messages"][-1].content
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return {
        "messages": state["messages"],
        "context": context
    }


def chatbot_node(state: State) -> State:
    prompt = f"""
You are a precise accounting information assistant for GL.
Use only the provided financial documentation to answer questions factually.
If information isn't available, respond professionally with:
"I don't have that specific information in our records - please contact our accounting team for assistance."

Answer requirements:
1. Provide only accounting/finance facts from the context
2. Never add conversational fluff or invitations to ask more
3. Quote exact figures/dates when available
4. Reference relevant forms/regulations where applicable
5. Keep answers under 3 sentences unless technical details require more

For unrelated questions:
"That question falls outside our accounting scope - please contact client services."

Context: {state['context']}
Question: {state['messages'][-1].content}
Concise accounting answer:"""


    response = llm.invoke([HumanMessage(content=prompt)])
    return {
        "messages": state["messages"] + [response],
        "context": state["context"]
    }


graph_builder = StateGraph(State)
graph_builder.add_node("retrieve", retrieve_node)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "chatbot")
graph_builder.add_edge("chatbot", END)
memory = MemorySaver()

graph_with_memory = graph_builder.compile(checkpointer=memory)




In [18]:
config = {
        "configurable": {
            "thread_id": "1"
        }
    }
output= graph_with_memory.invoke({"messages": [HumanMessage(content="What is GL?")]}, config=config)
print(output["messages"][-1].content)

 GL refers to General Ledger, which is an accounting tool designed to manage and track all financial transactions, automate journal entries, generate reports, and ensure compliance with accounting standards.


In [17]:
output= graph_with_memory.invoke({"messages": [HumanMessage(content="What did I just tell you? ")]}, config=config)
print(output["messages"][-1].content)

 The consistency principle states that an adopted accounting method should be used consistently over time for accurate comparison, and the materiality concept says only significant items affecting financial decisions should be reported, allowing for practical precision in records.


In [ ]:
# First message
output1 = graph_with_memory.invoke(
    {"messages": [HumanMessage(content="Hi, my name is Muhammad Ahmad.")]},
    config=config
)
print(output1["messages"][-1].content)

# Second message — memory will kick in!
output2 = graph_with_memory.invoke(
    {"messages": [HumanMessage(content="What did I just tell you?")]},
    config=config
)
print(output2["messages"][-1].content)


 Hello Muhammad Ahmad! In the provided context, a ledger is a book or digital record where all journal entries are posted to individual accounts. It helps track the balance of each account and prepares data for the trial balance and financial statements. A trial balance lists all the ledger account balances at a specific point in time to verify the arithmetic accuracy of bookkeeping by ensuring that total debits equal total credits. I hope this answers your question! If you have any other questions, feel free to ask!
 You just told me that a credit note is issued when goods are returned or overcharged, reducing the amount owed by the customer and adjusting the accounts receivable balance. A debit note is issued to request a credit for returned goods or billing adjustments. It informs the seller about the amount being claimed back.
